## Regularization

#### Table of contents

* [Overview](#Overview)
* [Setup](#Setup)
* [Lasso](#Lasso)

# Overview

This script covers how to implement two techniques:
1. Regularization
2. Cross-validation

Regularization is a shrinkage estimator. 
By adding an additional constraint on the optimization problem, the coefficients will be smaller than their non-constrained counterparts.
With an absolute constraint, it can be used to **select** features by setting coefficients to zero.
The shrinkage strength is determined by a hyperpameter $\lambda$, where $\lambda = 0$ is no shrinkage.

Cross-validation is a ubiquitous machine learning technique used **tune** hyperparameters.
Where train-test splits are used to compare across model classes, cross-validation is used for comparing within model classes.
See the lecture slides for more details.

***
# Setup
[TOP](#Regularization)

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import linear_model as lm

In [2]:
df = pd.read_pickle('C:/Users/johnj/Documents/Data/aml in econ 02 spring 2021/class data/class_data.pkl')
df.columns

Index(['pct_d_rgdp', 'urate_bin', 'pos_net_jobs', 'emp_estabs',
       'estabs_entry_rate', 'estabs_exit_rate', 'pop', 'pop_pct_black',
       'pop_pct_hisp', 'lfpr', 'density', 'year'],
      dtype='object')

In [3]:
df_prepped = df.drop(columns = ['urate_bin', 'year']).join([
    pd.get_dummies(df['urate_bin'], drop_first = True),
    pd.get_dummies(df.year, drop_first = True)    
])

In [4]:
y = df_prepped['pct_d_rgdp']
x = df_prepped.drop(columns = 'pct_d_rgdp')

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 2/3, random_state = 490)

x_train_std = x_train.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)
x_test_std  = x_test.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)

x_train_std = sm.add_constant(x_train_std)
x_test_std  = sm.add_constant(x_test_std)
x_train     = sm.add_constant(x_train)
x_test      = sm.add_constant(x_test)

###################################################
# Be careful if you have a large data set.        #
# We have created ~4 copies of the original data: #
#                                                 #
# 0. df                                           #
# 1. df_prepped                                   #
# 2. x, y                                         #
# 3. x_train, x_test, y_train, y_test             #
# 4. x_train_std, x_test_std                      #
#                                                 #
# you may run out of memeory                      #
###################################################

***
# Lasso
[TOP](#Regularization)

`statsmodels` fits an elastic-net constraint, which is a hybrid of Lasso and Ridge regression. 
It has the penalty:
$$
\alpha \left[ (1-L1\_wt)||\beta||_2 + L1\_wt||\beta||_1 \right]
$$

In [5]:
fit_ridge = sm.OLS(y_train, x_train_std).fit_regularized(alpha = 10, L1_wt = 0)
# note: there is no fit.summary()
fit_ridge.params

array([ 0.18029875,  0.07878553, -0.02016176,  0.09040581, -0.0217023 ,
       -0.0008972 , -0.0352795 ,  0.030556  ,  0.06993904, -0.00115653,
        0.05257679, -0.00836787,  0.02187259,  0.01610148,  0.01629544,
        0.05894326, -0.01071656, -0.02457684, -0.05852966,  0.01871018,
        0.00076798, -0.02270842,  0.0151311 , -0.0134313 , -0.00444493,
       -0.0382363 , -0.00899301,  0.00820053])

In [6]:
fit_lasso = sm.OLS(y_train, x_train_std).fit_regularized(alpha = 10, L1_wt = 1)
# note: there is no fit.summary()
fit_lasso.params

const                0.0
pos_net_jobs         0.0
emp_estabs           0.0
estabs_entry_rate    0.0
estabs_exit_rate     0.0
pop                  0.0
pop_pct_black        0.0
pop_pct_hisp         0.0
lfpr                 0.0
density              0.0
lower                0.0
similar              0.0
2003                 0.0
2004                 0.0
2005                 0.0
2006                 0.0
2007                 0.0
2008                 0.0
2009                 0.0
2010                 0.0
2011                 0.0
2012                 0.0
2013                 0.0
2014                 0.0
2015                 0.0
2016                 0.0
2017                 0.0
2018                 0.0
dtype: float64

***
# Cross-Validation 
[TOP](#Regularization)

We are using a specific type of k-fold cross-validation: grid search.
When performing grid search cross-validation, you always want to ensure that your selected hyperparameters are an interior point of the grid. 
Otherwise, you have no selected the optimal solution.

The grid search function is from `sklearn`.
Consequently, we need to use functions from the `linear_model` module: https://scikit-learn.org/stable/modules/linear_model.html.

In [ ]:
fit_lasso = sm.OLS(y_train, x_train_std).fit_regularized(alpha = 10, L1_wt = 1)
# note: there is no fit.summary()

In [7]:
param_grid = [
    {'alpha': 10**np.linspace(-2, 2, num = 20)}
]

# We are manually supplying an intercept
# and standardized (not normalized) the features
cv_lasso = lm.Lasso(fit_intercept = False, normalize = False,
                    random_state = 490)
grid_search = GridSearchCV(cv_lasso, param_grid, cv = 5,
                         scoring = 'neg_root_mean_squared_error')
grid_search.fit(x_train_std, y_train)
print(grid_search.best_params_)
best = grid_search.best_params_['alpha']
best

{'alpha': 0.01}


0.01

This is an interior point ($10^{-2} = 0.01$). 
We need to adjust our grid search.

In [8]:
param_grid = [
    {'alpha': 10**np.linspace(-5, -2, num = 20)}
]

cv_lasso = lm.Lasso(fit_intercept = False, normalize = False,
                    random_state = 490)

grid_search = GridSearchCV(cv_lasso, param_grid, cv = 5,
                         scoring = 'neg_root_mean_squared_error')
grid_search.fit(x_train_std, y_train)
best = grid_search.best_params_['alpha']
best

0.003359818286283781

***
Once we have identified the best value of $\alpha$, then the next step is to estimate the full model. 
We will use `statsmodels` for this task.

In [9]:
fit_lasso_tuned = sm.OLS(y_train, x_train_std).fit_regularized(L1_wt = 1, alpha = best)
fit_lasso_tuned.params

const                1.979926
pos_net_jobs         0.555051
emp_estabs          -0.175141
estabs_entry_rate    0.896385
estabs_exit_rate    -0.519264
pop                 -0.102140
pop_pct_black        0.014655
pop_pct_hisp         0.304292
lfpr                 0.489863
density             -0.005985
lower                0.601362
similar              0.250107
2003                 0.000000
2004                -0.069857
2005                 0.063817
2006                 0.436949
2007                -0.323667
2008                -0.354894
2009                -0.545083
2010                 0.168481
2011                -0.088483
2012                -0.383047
2013                 0.028383
2014                -0.309390
2015                -0.230910
2016                -0.581017
2017                -0.211965
2018                 0.000000
dtype: float64

***
Remember, a regularized model is biased.
We will fit 

- a non-regularized standardized model excluding the features that are zero 
- a non-regularized, non-standardized model excluding the features that are zero

In [10]:
beta = fit_lasso_tuned.params
beta.index[beta == 0]

Index([2003, 2018], dtype='object')

In [11]:
x_train_std_trim = x_train_std.loc[:, ~x_train_std.columns.isin(beta.index[beta == 0])]
x_test_std_trim = x_test_std.loc[:, ~x_test_std.columns.isin(beta.index[beta == 0])]

In [12]:
fit_std_final = sm.OLS(y_train, x_train_std_trim).fit()
fit_std_final.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.041      
Dependent Variable: pct_d_rgdp       AIC:                246977.3317
Date:               2021-02-25 14:24 BIC:                247196.5337
No. Observations:   33889            Log-Likelihood:     -1.2346e+05
Df Model:           25               F-statistic:        58.31      
Df Residuals:       33863            Prob (F-statistic): 4.48e-286  
R-squared:          0.041            Scale:              85.557     
--------------------------------------------------------------------
                     Coef.  Std.Err.    t     P>|t|   [0.025  0.975]
--------------------------------------------------------------------
const                1.9833   0.0502  39.4717 0.0000  1.8848  2.0818
pos_net_jobs         0.5576   0.0542  10.2891 0.0000  0.4514  0.6638
emp_estabs          -0.1799   0.0543  -3.3099 0.0009 -0.2864 -0.0733
estabs_entry_rate    0.8986   0.0597  15.0624 0.0000  0.7817  1.0155
estabs_exit_rate    -0.5245   0.0579  -9.0616 0.0000 -0.6380 -0.4111
pop                 -0.1051   0.0567  -1.8529 0.0639 -0.2164  0.0061
pop_pct_black        0.0231   0.0573   0.4031 0.6869 -0.0893  0.1355
pop_pct_hisp         0.3096   0.0522   5.9342 0.0000  0.2073  0.4118
lfpr                 0.4921   0.0624   7.8888 0.0000  0.3698  0.6144
density             -0.0089   0.0540  -0.1646 0.8693 -0.1147  0.0969
lower                0.6103   0.0684   8.9266 0.0000  0.4763  0.7443
similar              0.2587   0.0590   4.3821 0.0000  0.1430  0.3745
2004                -0.0797   0.0565  -1.4104 0.1584 -0.1904  0.0310
2005                 0.0635   0.0603   1.0525 0.2926 -0.0547  0.1816
2006                 0.4367   0.0613   7.1289 0.0000  0.3166  0.5568
2007                -0.3333   0.0566  -5.8880 0.0000 -0.4442 -0.2223
2008                -0.3639   0.0566  -6.4307 0.0000 -0.4748 -0.2530
2009                -0.5530   0.0580  -9.5330 0.0000 -0.6668 -0.4393
2010                 0.1662   0.0571   2.9088 0.0036  0.0542  0.2782
2011                -0.0976   0.0569  -1.7135 0.0866 -0.2092  0.0140
2012                -0.3924   0.0569  -6.8997 0.0000 -0.5039 -0.2810
2013                 0.0253   0.0568   0.4451 0.6563 -0.0861  0.1367
2014                -0.3193   0.0572  -5.5813 0.0000 -0.4315 -0.2072
2015                -0.2408   0.0569  -4.2324 0.0000 -0.3523 -0.1293
2016                -0.5912   0.0570 -10.3708 0.0000 -0.7029 -0.4794
2017                -0.2219   0.0570  -3.8938 0.0001 -0.3335 -0.1102
--------------------------------------------------------------------
Omnibus:            34604.616     Durbin-Watson:        1.994       
Prob(Omnibus):      0.000         Jarque-Bera (JB):     10667752.888
Skew:               4.483         Prob(JB):             0.000       
Kurtosis:           89.455        Condition No.:        3           
====================================================================

"""

In [14]:
x_train_trim = x_train.loc[:, ~x_train.columns.isin(beta.index[beta == 0])]
x_test_trim = x_test.loc[:, ~x_test.columns.isin(beta.index[beta == 0])]

In [15]:
fit_final = sm.OLS(y_train, x_train_trim).fit()
fit_final.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.041      
Dependent Variable: pct_d_rgdp       AIC:                246977.3317
Date:               2021-02-25 14:24 BIC:                247196.5337
No. Observations:   33889            Log-Likelihood:     -1.2346e+05
Df Model:           25               F-statistic:        58.31      
Df Residuals:       33863            Prob (F-statistic): 4.48e-286  
R-squared:          0.041            Scale:              85.557     
--------------------------------------------------------------------
                     Coef.  Std.Err.    t     P>|t|   [0.025  0.975]
--------------------------------------------------------------------
const               -2.6168   0.5422  -4.8265 0.0000 -3.6795 -1.5541
pos_net_jobs         1.1235   0.1092  10.2891 0.0000  0.9094  1.3375
emp_estabs          -0.0377   0.0114  -3.3099 0.0009 -0.0600 -0.0154
estabs_entry_rate    0.2968   0.0197  15.0624 0.0000  0.2582  0.3355
estabs_exit_rate    -0.2050   0.0226  -9.0616 0.0000 -0.2493 -0.1607
pop                 -0.0000   0.0000  -1.8529 0.0639 -0.0000  0.0000
pop_pct_black        0.0016   0.0039   0.4031 0.6869 -0.0061  0.0093
pop_pct_hisp         0.0236   0.0040   5.9342 0.0000  0.0158  0.0314
lfpr                 0.0441   0.0056   7.8888 0.0000  0.0332  0.0551
density             -0.0000   0.0000  -0.1646 0.8693 -0.0001  0.0001
lower                1.2577   0.1409   8.9266 0.0000  0.9815  1.5338
similar              0.6687   0.1526   4.3821 0.0000  0.3696  0.9678
2004                -0.3401   0.2411  -1.4104 0.1584 -0.8126  0.1325
2005                 0.2685   0.2551   1.0525 0.2926 -0.2315  0.7686
2006                 1.8407   0.2582   7.1289 0.0000  1.3346  2.3468
2007                -1.4058   0.2388  -5.8880 0.0000 -1.8738 -0.9378
2008                -1.5510   0.2412  -6.4307 0.0000 -2.0237 -1.0783
2009                -2.3392   0.2454  -9.5330 0.0000 -2.8201 -1.8582
2010                 0.7026   0.2415   2.9088 0.0036  0.2291  1.1760
2011                -0.4109   0.2398  -1.7135 0.0866 -0.8809  0.0591
2012                -1.6752   0.2428  -6.8997 0.0000 -2.1510 -1.1993
2013                 0.1077   0.2420   0.4451 0.6563 -0.3667  0.5821
2014                -1.3413   0.2403  -5.5813 0.0000 -1.8123 -0.8703
2015                -1.0404   0.2458  -4.2324 0.0000 -1.5222 -0.5586
2016                -2.5181   0.2428 -10.3708 0.0000 -2.9940 -2.0422
2017                -0.9448   0.2426  -3.8938 0.0001 -1.4203 -0.4692
--------------------------------------------------------------------
Omnibus:            34604.616     Durbin-Watson:        1.994       
Prob(Omnibus):      0.000         Jarque-Bera (JB):     10667752.888
Skew:               4.483         Prob(JB):             0.000       
Kurtosis:           89.455        Condition No.:        3896078     
====================================================================
* The condition number is large (4e+06). This might indicate
strong multicollinearity or other numerical problems.
"""

***
Lastly, we will test the models

In [16]:
rmse_null = np.sqrt(np.mean(  (y_test - np.mean(y_train))**2  ))
rmse_null

9.403229309446852

In [17]:
rmse_lasso = np.sqrt(np.mean(  (y_test - fit_lasso_tuned.predict(x_test_std))**2  ))
print(rmse_lasso)
round((rmse_lasso - rmse_null)/rmse_null*100, 2)

9.216937110787729


-1.98

In [18]:
rmse_std_final = np.sqrt(np.mean(  (y_test - fit_std_final.predict(x_test_std_trim))**2  ))
print(rmse_std_final)
round((rmse_std_final - rmse_null)/rmse_null*100, 2)

9.216949429609352


-1.98

In [19]:
rmse_final = np.sqrt(np.mean(  (y_test - fit_final.predict(x_test_trim))**2  ))
print(rmse_final)
round((rmse_final - rmse_null)/rmse_null*100, 2)

9.21691435737444


-1.98

Backward selection gave us a 1.37% reduction in RMSE. 

# What does ^ tell you about statistical significance for predictive power?